In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

In [ ]:
d2 = pd.read_csv(r"D:\datascience\Singapore_resale_price_project\ResaleFlatPricesBasedonApprovalDate2000Feb2012.csv")
d1 = pd.read_csv(r"D:\datascience\Singapore_resale_price_project\ResaleFlatPricesBasedonApprovalDate19901999.csv")
d4 = pd.read_csv(r"D:\datascience\Singapore_resale_price_project\ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv")
d5 = pd.read_csv(r"D:\datascience\Singapore_resale_price_project\ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv")
d3 = pd.read_csv(r"D:\datascience\Singapore_resale_price_project\ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv")

In [ ]:
d4

In [ ]:
d5.isna().sum()

In [ ]:
df = pd.concat((d1, d2, d3, d4, d5), axis = 0)
df

In [ ]:
df.isnull().sum()

In [ ]:
df1 = df.dropna()
df1

In [ ]:
df1.isnull().sum()

In [ ]:
df1['remaining_lease_year'] = 99-(2024- df1['lease_commence_date'])
df1 = df1.drop('remaining_lease' , axis = 1, inplace= False)

df1['flat_type'] = df1['flat_type'].replace('MULTI GENERATION', 'MULTI-GENERATION')

df1['flat_model'] = df1['flat_model'].replace('IMPROVED', 'Improved')
df1['flat_model'] = df1['flat_model'].replace('IMPROVED-MAISONETTE', 'Improved-Maisonette')
df1['flat_model'] = df1['flat_model'].replace('2-ROOM', '2-room')
df1['flat_model'] = df1['flat_model'].replace('PREMIUM APARTMENT', 'Premium Apartment')
df1['flat_model'] = df1['flat_model'].replace('MAISONETTE', 'Maisonette')
df1['flat_model'] = df1['flat_model'].replace('MODEL A-MAISONETTE', 'Model A-Maisonette')
df1['flat_model'] = df1['flat_model'].replace('MODEL A', 'Model A')
df1['flat_model'] = df1['flat_model'].replace('NEW GENERATION', 'New Generation')
df1['flat_model'] = df1['flat_model'].replace('STANDARD', 'Standard')
df1['flat_model'] = df1['flat_model'].replace('APARTMENT', 'Apartment')
df1['flat_model'] = df1['flat_model'].replace('MULTI GENERATION', 'Multi Generation')
df1['flat_model'] = df1['flat_model'].replace('SIMPLIFIED', 'Simplified')

In [ ]:
df1['flat_type'].unique()
df1['flat_model'].unique()
df1['town'].unique()

In [ ]:
df1['flat_type_enc'] = OrdinalEncoder().fit_transform(df1[['flat_type']])

df1['flat_model_enc'] = OrdinalEncoder().fit_transform(df1[['flat_model']])

df1['town_enc'] = OrdinalEncoder().fit_transform(df1[['town']])

In [ ]:
df1['flat_type_enc'].unique()
df1['flat_model_enc'].unique()
df1['town_enc'].unique()

In [ ]:
def median_val(val):
    k = [float(i) for i in val.split("TO")]
    m = statistics.median(k)
    return m

df1['story_range_med'] = df1['storey_range'].apply(median_val)

In [ ]:
df1.isnull().sum()

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:
columns = ['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range',
       'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price',
       'remaining_lease', 'remaining_lease_year']

In [ ]:
df2 = df1.drop_duplicates()

In [ ]:
df2

In [ ]:
df.to_csv(r"D:\datascience\Singapore_resale_price_project\data.csv")

In [ ]:
plt.figure(figsize= (15,6))
sns.barplot(x = 'town', y = 'resale_price', data = df2)
plt.title('Average Price Range per Town')
plt.xlabel('Town')
plt.ylabel('Resale Price')
plt.xticks(rotation = 45)
plt.show()

In [ ]:
plt.figure(figsize= (15,6))
sns.barplot(x = 'flat_type', y = 'resale_price', data = df2)
plt.title('Average Price Range per Type')
plt.xlabel('Flat Type')
plt.ylabel('Resale Price')
plt.xticks(rotation = 45)
plt.show()

In [ ]:
df2.describe().T

In [ ]:
a = df2[df2['flat_type'] == '3 ROOM']['resale_price']
a.mean()

In [ ]:
def plot(df, columns):
    for i in columns:
        plt.figure(figsize=(20,5))
        
        plt.subplot(1,3,1)
        sns.boxplot(df[i])
        plt.title(f'Box Plot for {i}')

        plt.subplot(1,3,2)
        sns.histplot(df[i], kde=True, bins=50)
        plt.title(f'Distribution Plot for {i}')

        plt.subplot(1,3,3)
        sns.violinplot(df[i])
        plt.title(f'Violin Plot for {i}')
        
        plt.show()

In [ ]:
columns = ['floor_area_sqm', 'lease_commence_date', 'resale_price', 'remaining_lease_year', 'flat_type_enc', 'flat_model_enc', 'town_enc', 'story_range_med']
plot(df2, columns)

In [ ]:
df3 = df2.copy()

df3['story_range_log'] = np.log(df3['story_range_med'])
df3['remaining_lease_log'] = np.log(df3['remaining_lease_year'])
df3['resale_price_log'] = np.log(df3['resale_price'])
df3['lease_commence_date_log'] = np.log(df3['lease_commence_date'])
df3['floor_area_log'] = np.log(df3['floor_area_sqm'])

In [ ]:
columns = ['floor_area_log', 'lease_commence_date_log', 'resale_price_log', 'remaining_lease_log', 'flat_type_enc', 'flat_model_enc', 'town_enc', 'story_range_log']
plot(df3, columns)

In [ ]:
z_scores = df3.select_dtypes(include=np.number).apply(lambda x: np.abs((x - x.mean()) / x.std()))

threshold = 3

df4 = df3[(z_scores < threshold).all(axis=1)]

df4

In [ ]:
plot(df4, columns)

In [ ]:
def regression(x, y, algorithm):
     
    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

    model = algorithm().fit(x_train, y_train)
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)


    accuracy_metrics = {'algorithm': algorithm.__name__,
                        'R2_train' : r2_train,
                        'R2_test'  : r2_test}

    return accuracy_metric

In [ ]:
# col = ['floor_area_log', 'lease_commence_date_log', 'resale_price_log', 'town_enc', 'story_range_log']
columns = ['floor_area_log', 'lease_commence_date_log', 'remaining_lease_log', 'flat_type_enc', 'flat_model_enc', 'town_enc', 'story_range_log', 'resale_price_log']
dff = df4[columns]

In [ ]:
columns = ['floor_area_log',  'remaining_lease_log', 'flat_type_enc', 'flat_model_enc', 'town_enc', 'story_range_log', 'resale_price_log']
dff = df4[columns]

In [ ]:
dff

In [ ]:
dff.describe().T

In [ ]:
x = dff.drop('resale_price_log', axis = 1)
y = dff['resale_price_log']
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
x

In [ ]:
corr = dff.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
print(regression(x, y, DecisionTreeRegressor))
print(regression(x, y, ExtraTreesRegressor))
print(regression(x, y, RandomForestRegressor))
print(regression(x, y, AdaBoostRegressor))
print(regression(x, y, GradientBoostingRegressor))
print(regression(x, y, XGBRegressor))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

xgb = XGBRegressor()

grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='r2')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_xgb = XGBRegressor(**best_params)
best_xgb.fit(X_train, y_train)

y_pred = best_xgb.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("Best Parameters:", best_params)
print("Best R2 Score:", best_score)
print("R2 Score on Test Set:", r2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

xgb = XGBRegressor()

grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='r2')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_xgb = XGBRegressor(**best_params)
best_xgb.fit(X_train, y_train)

y_pred = best_xgb.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

metrics_r = {'R2': r2,
           'Mean Absolute Error': mae,
           'Mean Squared Error': mse,
           'Root Mean Squared Error': rmse}

metrics_r

In [ ]:
X_test

In [ ]:
test_data = np.array([[4.094345, 7.593878, 4.110874, 2.0, 5.0, 0.0, 2.079442]])

test_data_scaled = scaler.transform(test_data)

y_pred = best_xgb.predict(test_data_scaled)

np.exp(y_pred[0])

In [ ]:
test_data = np.array([[np.log(60), np.log(61), 2, 5, 0, np.log(8)]])

test_data_scaled = scaler.transform(test_data)

y_pred = best_xgb.predict(test_data_scaled)

np.exp(y_pred[0])

In [ ]:
np.exp(13.449019)

In [ ]:
test_data = np.array([[4.094345, 4.110874, 2.0, 5.0, 7.0, 2.079442]])

test_data_scaled = scaler.transform(test_data)

y_pred = best_xgb.predict(test_data_scaled)

np.exp(y_pred[0])

In [ ]:
with open(r"D:\datascience\Singapore_resale_price_project\regression_model_pkl", 'wb') as f:
    pickle.dump(best_xgb, f)

with open(r"D:\datascience\Singapore_resale_price_project\scaler.pkl", 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
with open(r"D:\datascience\Singapore_resale_price_project\regression_model_pkl", 'rb') as f:
    model = pickle.load(f)

with open(r"D:\datascience\Singapore_resale_price_project\scaler.pkl", 'rb') as f:
    scaler = pickle.load(f)

In [ ]:
test1 = np.array([[np.log(55), np.log(63), 2, 5, 7, np.log(5)]])

t1 = scaler.fit_transform(test1)

y_pred1 = model.predict(t1)

print(*np.exp(y_pred1))

In [ ]:
test_data = np.array([[4.094345, 4.110874, 3.0, 5.0, 7.0, 2.079442]])

test_data_scaled = scaler.transform(test_data)

y_pred = model.predict(test_data_scaled)

np.exp(y_pred[0])